# LAB 07: Lakeflow Declarative Pipeline

**Duration:** ~45 min | **Day:** 3 | **Difficulty:** Advanced

> *"Build a Medallion pipeline: Bronze (streaming), Silver (validated), Gold (aggregated) using Lakeflow SQL declarations."*

This lab has two parts:
1. **SQL files** (created separately) -- pipeline declarations
2. **This notebook** -- verification and exploration of pipeline results

## Setup

In [ ]:
%run ../../setup/00_setup

---
## Task 1: Write Bronze Declaration

Complete the SQL below to create a Bronze streaming table from JSON files.

**This SQL would go in a pipeline SQL file.** Here we practice the syntax.

In [ ]:
# Practice: what the Bronze SQL declaration looks like
# (This won't execute outside of a Lakeflow pipeline, but verify the syntax)

bronze_sql = f"""
-- TODO: Complete the Bronze declaration
CREATE OR REFRESH ________ TABLE bronze_orders
AS SELECT * 
FROM STREAM ________('{DATASET_PATH}/orders/stream', format => 'json');
"""

print("Bronze SQL declaration:")
print(bronze_sql)

In [ ]:
# -- Validation --
assert "STREAMING" in bronze_sql.upper(), "Should use STREAMING TABLE"
assert "read_files" in bronze_sql.lower(), "Should use read_files()"
print("Task 1 OK: Bronze declaration syntax correct")

---
## Task 2: Write Silver Declaration with Expectations

Complete the Silver layer with data quality constraints.

In [ ]:
# TODO: Complete Silver SQL with expectations
silver_sql = """
CREATE OR REFRESH STREAMING TABLE silver_orders (
    CONSTRAINT valid_id EXPECT (order_id IS NOT NULL) ON VIOLATION ________,
    CONSTRAINT positive_amount EXPECT (total_price > 0) ON VIOLATION ________
)
AS SELECT 
    order_id,
    customer_id,
    product_id,
    CAST(quantity AS INT) AS quantity,
    CAST(total_price AS DOUBLE) AS total_price,
    CAST(order_date AS DATE) AS order_date,
    payment_method,
    store_id,
    current_timestamp() AS processed_at
FROM STREAM(LIVE.bronze_orders);
"""

print("Silver SQL declaration:")
print(silver_sql)

In [ ]:
# -- Validation --
assert "CONSTRAINT" in silver_sql.upper(), "Should have CONSTRAINT declarations"
assert "DROP ROW" in silver_sql.upper(), "Should use ON VIOLATION DROP ROW"
assert "LIVE.bronze" in silver_sql.lower(), "Should reference LIVE.bronze_orders"
print("Task 2 OK: Silver declaration with expectations correct")

---
## Task 3: Write Gold Declaration

Create a Materialized View for daily revenue summary.

In [ ]:
# TODO: Complete Gold declaration
gold_sql = """
CREATE OR REFRESH ________ ________ gold_daily_revenue
AS SELECT 
    order_date,
    SUM(total_price) AS total_revenue,
    COUNT(*) AS total_orders,
    AVG(total_price) AS avg_order_value
FROM LIVE.silver_orders
GROUP BY order_date
ORDER BY order_date;
"""

print("Gold SQL declaration:")
print(gold_sql)

In [ ]:
# -- Validation --
assert "MATERIALIZED VIEW" in gold_sql.upper(), "Gold should use MATERIALIZED VIEW"
assert "LIVE.silver" in gold_sql.lower(), "Should reference LIVE.silver_orders"
print("Task 3 OK: Gold Materialized View declaration correct")

---
## Task 4: Compare STREAMING TABLE vs MATERIALIZED VIEW

Fill in the comparison table (markdown exercise).

| Feature | STREAMING TABLE | MATERIALIZED VIEW |
|---------|----------------|-------------------|
| Processing mode | ________ | ________ |
| Best for | ________ | ________ |
| Read from source | STREAM(LIVE.x) | LIVE.x |
| Supports expectations | Yes | Yes |

---
## Task 5: Verify Pipeline Results (after running the pipeline)

After creating and running the pipeline via the UI, query the results here.

In [ ]:
# TODO: After running the pipeline, uncomment and verify:

# display(spark.sql(f"SELECT COUNT(*) as cnt FROM {CATALOG}.bronze.bronze_orders"))
# display(spark.sql(f"SELECT COUNT(*) as cnt FROM {CATALOG}.silver.silver_orders"))
# display(spark.sql(f"SELECT * FROM {CATALOG}.gold.gold_daily_revenue ORDER BY order_date"))

---
## Task 6: Check Pipeline Event Log

Query the pipeline event log to see data quality metrics.

In [ ]:
# TODO: After running the pipeline, uncomment to check event log:

# display(spark.sql("""
#     SELECT timestamp, details:flow_progress:data_quality:expectations
#     FROM event_log(TABLE({CATALOG}.silver.silver_orders))
#     WHERE details:flow_progress:data_quality IS NOT NULL
# """))

---
## Lab Complete!

You have:
- Written Bronze STREAMING TABLE declarations
- Written Silver declarations with data quality expectations
- Written Gold MATERIALIZED VIEW declarations
- Understood ST vs MV differences
- (If pipeline ran) Verified results and checked data quality metrics

> **Exam Tip:** In Lakeflow SQL, use `LIVE.table_name` to reference other tables in the same pipeline. The `LIVE.` prefix is pipeline-scoped -- it doesn't appear in Unity Catalog.

> **Next:** LAB 08 - Lakeflow Jobs & Orchestration